In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define the LeNet-5 architecture
class LeNet5(nn.Module):
    def _init_(self):
        super(LeNet5, self)._init_()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the SVHN dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Initialize the LeNet-5 model
lenet5_model = LeNet5()

# Load pretrained weights (if available)
try:
    lenet5_model.load_state_dict(models.lenet5(pretrained=True).state_dict())
    print("Loaded pretrained weights for LeNet-5.")
except:
    print("Pretrained weights not found for LeNet-5. Training from scratch...")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet5_model.parameters(), lr=0.001)

# Train the model
lenet5_model.train()
for epoch in range(5):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = lenet5_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[LeNet-5, Epoch {epoch + 1}, Batch {i + 1}]: Loss = {running_loss / 100:.3f}")
            running_loss = 0.0

# Evaluate the model
lenet5_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = lenet5_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of LeNet-5: {100 * correct/total:.2f}%")


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load the SVHN dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match VGG's expected input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load only 25% of the dataset
trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
trainset_subset = torch.utils.data.Subset(trainset, range(len(trainset) // 4))  # Use only 25% of the training data
testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)
testset_subset = torch.utils.data.Subset(testset, range(len(testset) // 4))  # Use only 25% of the test data

trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset_subset, batch_size=64, shuffle=False)

# Define function to train and evaluate model
def train_and_evaluate(model):
    # Modify the last fully connected layer
    num_classes = 10
    model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    model.train()
    for epoch in range(5):  # Adjust the number of epochs as needed
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 100 == 99:  # Print every 100 mini-batches
                print(f"[{model._class.name_}, Epoch {epoch + 1}, Batch {i + 1}]: Loss = {running_loss / 100:.3f}")
                running_loss = 0.0

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy of {model._class.name_}: {100 * correct / total:.2f}%")

# Load pretrained VGG model
vgg_model = models.vgg16(pretrained=True)

# Train and evaluate the model
train_and_evaluate(vgg_model)


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 85.7MB/s]


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load the SVHN dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match AlexNet's expected input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load only 25% of the dataset
trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
trainset_subset = torch.utils.data.Subset(trainset, range(len(trainset) // 4))  # Use only 25% of the training data
testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)
testset_subset = torch.utils.data.Subset(testset, range(len(testset) // 4))  # Use only 25% of the test data

trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset_subset, batch_size=64, shuffle=False)

# Define function to train and evaluate model
def train_and_evaluate(model):
    # Modify the last fully connected layer
    num_classes = 10
    model.classifier[6] = nn.Linear(4096, num_classes)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    model.train()
    for epoch in range(5):  # Adjust the number of epochs as needed
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 100 == 99:  # Print every 100 mini-batches
                print(f"[{model._class.name_}, Epoch {epoch + 1}, Batch {i + 1}]: Loss = {running_loss / 100:.3f}")
                running_loss = 0.0

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy of {model._class.name_}: {100 * correct / total:.2f}%")

# Load pretrained AlexNet model
alexnet_model = models.alexnet(pretrained=True)

# Train and evaluate the AlexNet model
train_and_evaluate(alexnet_model)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load the SVHN dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match ResNet's expected input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load only 25% of the dataset
trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
trainset_subset = torch.utils.data.Subset(trainset, range(len(trainset) // 4))  # Use only 25% of the training data
testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)
testset_subset = torch.utils.data.Subset(testset, range(len(testset) // 4))  # Use only 25% of the test data

trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset_subset, batch_size=64, shuffle=False)

# Define function to train and evaluate model
def train_and_evaluate(model):
    # Modify the last fully connected layer
    num_classes = 10
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    model.train()
    for epoch in range(5):  # Adjust the number of epochs as needed
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 100 == 99:  # Print every 100 mini-batches
                print(f"[{model._class.name_}, Epoch {epoch + 1}, Batch {i + 1}]: Loss = {running_loss / 100:.3f}")
                running_loss = 0.0

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy of {model._class.name_}: {100 * correct / total:.2f}%")

# Load pretrained ResNet models
resnet18_model = models.resnet18(pretrained=True)
resnet50_model = models.resnet50(pretrained=True)
resnet101_model = models.resnet101(pretrained=True)

# Train and evaluate each model
for model in [resnet18_model, resnet50_model, resnet101_model]:
    train_and_evaluate(model)
